<a href="https://colab.research.google.com/github/yoonju977/Personalize_Medic_chatbot/blob/main/M_Bert_pilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#라이브러리
!pip install -U torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121
!pip uninstall -y cuml-cu12 sklearn-compat scikit-learn || true
!pip install -U "transformers==4.38.2" sentence-transformers accelerate datasets evaluate "scikit-learn==1.6.1" umap-learn
!pip install -U "numpy<2.0"

Looking in indexes: https://download.pytorch.org/whl/cu121
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
  Using cached sentence_transformers-5.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-4.0.2-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-4.0.1-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-4.0.0-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using

In [2]:
import torch, sklearn, transformers, numpy, torchvision, torchaudio
print("torch:", torch.__version__, "| cuda:", torch.version.cuda, "| gpu:", torch.cuda.is_available())
print("torchvision:", torchvision.__version__, "| torchaudio:", torchaudio.__version__)
print("scikit-learn:", sklearn.__version__)
print("transformers:", transformers.__version__)
print("numpy:", numpy.__version__)

torch: 2.3.1+cu121 | cuda: 12.1 | gpu: True
torchvision: 0.18.1+cu121 | torchaudio: 2.3.1+cu121
scikit-learn: 1.6.1
transformers: 4.38.2
numpy: 1.26.4


In [4]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/Medic_chatbot/Medic_chatbot/Symptom2Disease.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb"
MODEL_NAME = "bert-base-multilingual-cased"  # mBERT (cased)

import os, random, numpy as np, torch
os.makedirs(OUTPUT_DIR, exist_ok=True)

random.seed(42); np.random.seed(42); torch.manual_seed(42)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(42)

print("GPU:", torch.cuda.is_available(), "| Output dir:", OUTPUT_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPU: True | Output dir: /content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb


In [7]:
import pandas as pd, re

text_col = "text"
label_col = "label"

df = pd.read_csv(DATA_PATH)
df[text_col] = df[text_col].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
df[label_col] = df[label_col].astype(str).str.strip()

# (옵션) 라벨 통일 맵핑 (필요 시 추가)
canon = {
    # 예: "Diabetes":"diabetes"
}
df[label_col] = df[label_col].map(lambda x: canon.get(x, x))

before = len(df)
df = df.drop_duplicates(subset=[text_col, label_col]).reset_index(drop=True)
print(f"[클린업] 완전 중복 제거: {before-len(df)} / {before} | 총 샘플: {len(df)}")
print(df[label_col].value_counts().sort_index())

[클린업] 완전 중복 제거: 47 / 1200 | 총 샘플: 1153
label
Acne                               46
Arthritis                          46
Bronchial Asthma                   49
Cervical spondylosis               49
Chicken pox                        49
Common Cold                        49
Dengue                             50
Dimorphic Hemorrhoids              41
Fungal infection                   50
Hypertension                       50
Impetigo                           50
Jaundice                           38
Malaria                            44
Migraine                           47
Pneumonia                          47
Psoriasis                          50
Typhoid                            50
Varicose Veins                     50
allergy                            50
diabetes                           50
drug reaction                      50
gastroesophageal reflux disease    48
peptic ulcer disease               50
urinary tract infection            50
Name: count, dtype: int64


In [1]:
!pip uninstall -y sentence-transformers
!pip install "sentence-transformers==2.6.1"

Found existing installation: sentence-transformers 3.1.1
Uninstalling sentence-transformers-3.1.1:
  Successfully uninstalled sentence-transformers-3.1.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 11.8 MB/s eta 0:00:00


In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np, torch

embed_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

embeddings = embed_model.encode(
    df[text_col].tolist(),
    batch_size=128,
    convert_to_numpy=True,
    show_progress_bar=True,
    device=embed_model.device
)

S = cosine_similarity(embeddings)
np.fill_diagonal(S, 0.0)

THRESH = 0.92  # 0.90~0.95 사이 튜닝

parent = list(range(len(df)))
def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x
def union(a,b):
    ra, rb = find(a), find(b)
    if ra != rb: parent[rb] = ra

n = len(df)
for i in range(n):
    idx = np.where(S[i] >= THRESH)[0]
    for j in idx:
        union(i, j)

group_id = [find(i) for i in range(n)]
roots = {r:i for i,r in enumerate(sorted(set(group_id)))}
df["group_id"] = [roots[r] for r in group_id]
print("총 그룹 수:", df["group_id"].nunique())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

총 그룹 수: 578


In [9]:
from sklearn.model_selection import StratifiedGroupKFold
from collections import Counter

LABELS = sorted(df[label_col].unique().tolist())
label2id = {l:i for i,l in enumerate(LABELS)}
id2label = {i:l for l,i in label2id.items()}

y = df[label_col].map(label2id).values
groups = df["group_id"].values
idx_all = np.arange(len(df))

def strat_group_split(X_index, y, groups, test_size=0.15, random_state=42):
    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=random_state)
    folds = list(sgkf.split(X_index, y, groups))
    target = int(len(X_index)*test_size)
    best = None
    for _, test_idx in folds:
        if best is None or abs(len(test_idx)-target) < abs(len(best)-target):
            best = test_idx
    test_idx = np.array(sorted(best))
    train_idx = np.array(sorted(list(set(X_index) - set(test_idx))))
    return train_idx, test_idx

# 1) test
train_idx, test_idx = strat_group_split(idx_all, y, groups, test_size=0.15, random_state=42)
# 2) val (train 내부에서 다시)
train_y, train_groups = y[train_idx], groups[train_idx]
sub_train_idx, val_idx = strat_group_split(train_idx, train_y, train_groups, test_size=0.15, random_state=123)
train_idx = sub_train_idx

def split_report(name, idx):
    cnt = Counter(df.iloc[idx][label_col])
    print(f"{name}: n={len(idx)} | 클래스수={len(cnt)}")
    print(dict(sorted(cnt.items())))

split_report("TRAIN", train_idx)
split_report("VAL", val_idx)
split_report("TEST", test_idx)

train_df = df.iloc[train_idx].reset_index(drop=True)
val_df   = df.iloc[val_idx].reset_index(drop=True)
test_df  = df.iloc[test_idx].reset_index(drop=True)

import os
train_path = os.path.join(OUTPUT_DIR, "train.csv")
val_path   = os.path.join(OUTPUT_DIR, "val.csv")
test_path  = os.path.join(OUTPUT_DIR, "test.csv")

train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)
test_df.to_csv(test_path, index=False)

print("Saved:", train_path, "|", val_path, "|", test_path)

TRAIN: n=849 | 클래스수=24
{'Acne': 36, 'Arthritis': 37, 'Bronchial Asthma': 33, 'Cervical spondylosis': 38, 'Chicken pox': 37, 'Common Cold': 29, 'Dengue': 36, 'Dimorphic Hemorrhoids': 40, 'Fungal infection': 32, 'Hypertension': 43, 'Impetigo': 29, 'Jaundice': 29, 'Malaria': 28, 'Migraine': 45, 'Pneumonia': 38, 'Psoriasis': 24, 'Typhoid': 35, 'Varicose Veins': 31, 'allergy': 37, 'diabetes': 47, 'drug reaction': 35, 'gastroesophageal reflux disease': 38, 'peptic ulcer disease': 40, 'urinary tract infection': 32}
VAL: n=140 | 클래스수=17
{'Acne': 10, 'Arthritis': 7, 'Cervical spondylosis': 10, 'Chicken pox': 6, 'Common Cold': 9, 'Dengue': 5, 'Fungal infection': 12, 'Impetigo': 10, 'Jaundice': 2, 'Malaria': 16, 'Pneumonia': 2, 'Psoriasis': 14, 'Typhoid': 6, 'Varicose Veins': 9, 'allergy': 11, 'gastroesophageal reflux disease': 4, 'urinary tract infection': 7}
TEST: n=179 | 클래스수=22
{'Arthritis': 2, 'Bronchial Asthma': 16, 'Cervical spondylosis': 1, 'Chicken pox': 6, 'Common Cold': 13, 'Dengue': 1

In [1]:
!pip uninstall -y transformers sentence-transformers
!pip install -U "transformers==4.45.2" "sentence-transformers==3.2.1" "datasets>=2.20.0"

Found existing installation: transformers 4.45.2
Uninstalling transformers-4.45.2:
  Successfully uninstalled transformers-4.45.2
Found existing installation: sentence-transformers 3.2.1
Uninstalling sentence-transformers-3.2.1:
  Successfully uninstalled sentence-transformers-3.2.1
  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
  Using cached sentence_transformers-3.2.1-py3-none-any.whl.metadata (10 kB)
Using cached transformers-4.45.2-py3-none-any.whl (9.9 MB)
Using cached sentence_transformers-3.2.1-py3-none-any.whl (255 kB)


In [2]:
# 드라이브 마운트 & 경로
from google.colab import drive
drive.mount('/content/drive')

import os, numpy as np, torch
OUTPUT_DIR = "/content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb"
train_path = os.path.join(OUTPUT_DIR, "train.csv")
val_path   = os.path.join(OUTPUT_DIR, "val.csv")
test_path  = os.path.join(OUTPUT_DIR, "test.csv")
MODEL_NAME = "bert-base-multilingual-cased"
text_col, label_col = "text", "label"

# 라벨 맵핑은 train.csv에서 재구성
import pandas as pd
LABELS = sorted(pd.read_csv(train_path)[label_col].unique().tolist())
label2id = {l:i for i,l in enumerate(LABELS)}
id2label = {i:l for l,i in label2id.items()}
print("labels:", len(LABELS))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
labels: 24


In [3]:
# ---- 5단계: mBERT 파인튜닝 ----
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.utils.class_weight import compute_class_weight
import evaluate
from torch import nn

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch[text_col], padding="max_length", truncation=True, max_length=256)

dataset = load_dataset("csv", data_files={"train": train_path, "validation": val_path, "test": test_path})

def encode_label(example):
    example["label_id"] = label2id[example[label_col]]
    return example

dataset = dataset.map(encode_label)
dataset = dataset.map(tokenize_fn, batched=True)
dataset = dataset.rename_column("label_id", "labels")
dataset = dataset.remove_columns([c for c in dataset["train"].column_names if c not in ["input_ids","attention_mask","labels"]])

# 클래스 가중치
y_train = dataset["train"]["labels"]
class_weights = compute_class_weight(class_weight="balanced", classes=np.arange(len(LABELS)), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABELS))
model.config.label2id = label2id
model.config.id2label = id2label

class WeightedCELossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**{k: v for k,v in inputs.items() if k!="labels"})
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision_macro": precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall_macro": recall.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

args = TrainingArguments(
    output_dir=os.path.join(OUTPUT_DIR, "checkpoints"),
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=True,
    logging_steps=50,
    report_to="none",
    save_total_limit=2,
)

trainer = WeightedCELossTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro
1,3.029700,2.723868,0.228571,0.144971,0.209463,0.206521
2,2.518300,2.072781,0.700000,0.484487,0.525448,0.502515
3,1.995600,1.683136,0.814286,0.622640,0.644675,0.631645
4,1.713100,1.522916,0.864286,0.687407,0.690806,0.694423


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

TrainOutput(global_step=216, training_loss=2.2608186227303966, metrics={'train_runtime': 57.0594, 'train_samples_per_second': 59.517, 'train_steps_per_second': 3.786, 'total_flos': 446850820620288.0, 'train_loss': 2.2608186227303966, 'epoch': 4.0})

In [4]:
# ---- 6단계: 테스트 평가 & 예측 저장 ----
metrics = trainer.evaluate(dataset["test"])
print("Test metrics:", metrics)

import pandas as pd, numpy as np, os
test_logits = trainer.predict(dataset["test"]).predictions
test_preds = np.argmax(test_logits, axis=-1)
id2label_sorted = {v:k for k,v in label2id.items()}
pred_labels = [id2label_sorted[i] for i in test_preds]

raw_test = pd.read_csv(test_path)
raw_test["pred_label"] = pred_labels
save_pred_path = os.path.join(OUTPUT_DIR, "test_with_preds.csv")
raw_test.to_csv(save_pred_path, index=False)
print("Saved:", save_pred_path)

Test metrics: {'eval_loss': 1.9033857583999634, 'eval_accuracy': 0.6759776536312849, 'eval_f1_macro': 0.6380010259105696, 'eval_precision_macro': 0.6693926592095929, 'eval_recall_macro': 0.6843341441167528, 'eval_runtime': 0.2155, 'eval_samples_per_second': 830.45, 'eval_steps_per_second': 27.836, 'epoch': 4.0}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Saved: /content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb/test_with_preds.csv


In [5]:
save_dir = os.path.join(OUTPUT_DIR, "best_model_mb")
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print("Saved model to:", save_dir)

Saved model to: /content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb/best_model_mb


In [7]:
import os, pandas as pd, numpy as np
from sklearn.metrics import classification_report, confusion_matrix

pred_path = os.path.join(OUTPUT_DIR, "test_with_preds.csv")
dfp = pd.read_csv(pred_path)

# 라벨 인덱스화
y_true = [label2id[l] for l in dfp[label_col].tolist()]
y_pred = [label2id[l] for l in dfp["pred_label"].tolist()]

# 테스트에 실제로 존재하는 클래스만 리포트
present_ids = sorted(np.unique(y_true))
present_labels = [id2label[i] for i in present_ids]

print(classification_report(
    y_true, y_pred,
    labels=present_ids,
    target_names=present_labels,
    digits=3,
    zero_division=0
))

cm = confusion_matrix(y_true, y_pred, labels=present_ids)

                                 precision    recall  f1-score   support

                      Arthritis      1.000     1.000     1.000         2
               Bronchial Asthma      0.727     1.000     0.842        16
           Cervical spondylosis      0.167     1.000     0.286         1
                    Chicken pox      1.000     0.667     0.800         6
                    Common Cold      0.667     0.769     0.714        13
                         Dengue      1.000     0.300     0.462        10
          Dimorphic Hemorrhoids      1.000     1.000     1.000         1
               Fungal infection      1.000     1.000     1.000         6
                   Hypertension      0.875     1.000     0.933         7
                       Impetigo      1.000     0.818     0.900        11
                       Jaundice      0.875     1.000     0.933         7
                       Migraine      1.000     1.000     1.000         2
                      Pneumonia      0.500     0.3

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# 혼동 행렬 저장 경로
cm_path = os.path.join(OUTPUT_DIR, "confusion_matrix.png")

# seaborn heatmap으로 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=present_labels,
    yticklabels=present_labels
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.tight_layout()

# 이미지 저장
plt.savefig(cm_path, dpi=300)
plt.close()

print(f"혼동 행렬 저장 완료: {cm_path}")

혼동 행렬 저장 완료: /content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb/confusion_matrix.png


In [11]:
# === 자동 컬럼 감지 → 리포트/혼동행렬 저장 올인원 ===
import os
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

OUTPUT_DIR = "/content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb"
pred_csv   = os.path.join(OUTPUT_DIR, "test_with_preds.csv")

df = pd.read_csv(pred_csv)
print("[columns]", list(df.columns))

# 1) 컬럼 자동 감지
true_candidates = ["label", "true_label", "gold", "target", "y_true"]
pred_candidates = ["pred_label", "prediction", "pred", "y_pred", "label_pred"]

true_col = next((c for c in true_candidates if c in df.columns), None)
pred_col = next((c for c in pred_candidates if c in df.columns), None)

if true_col is None or pred_col is None:
    raise KeyError(
        f"True/Pred column not found.\n"
        f"Found columns: {list(df.columns)}\n"
        f"Expected one of true={true_candidates}, pred={pred_candidates}"
    )

y_true = df[true_col].astype(str).tolist()
y_pred = df[pred_col].astype(str).tolist()

# 2) 테스트에 실제로 등장/예측된 클래스 집합
present_labels = sorted(set(y_true) | set(y_pred))

# 3) classification_report 저장
report_dict = classification_report(
    y_true, y_pred,
    labels=present_labels,        # 문자열 라벨 직접 사용
    target_names=present_labels,  # 보고서에 질병명 그대로
    digits=3,
    zero_division=0,
    output_dict=True
)
report_df = pd.DataFrame(report_dict).transpose()
report_path = os.path.join(OUTPUT_DIR, "classification_report.csv")
report_df.to_csv(report_path, index=True, encoding="utf-8-sig")
print(f"[saved] {report_path}")

# 4) 혼동행렬 계산(정수 카운트)
cm = confusion_matrix(y_true, y_pred, labels=present_labels)
cm_csv_path = os.path.join(OUTPUT_DIR, "confusion_matrix_counts.csv")
pd.DataFrame(cm, index=[f"T_{l}" for l in present_labels],
                columns=[f"P_{l}" for l in present_labels]).to_csv(cm_csv_path, encoding="utf-8-sig")
print(f"[saved] {cm_csv_path}")

# 5) 혼동행렬 시각화(plt만 사용, PNG 저장)
plt.figure(figsize=(max(8, len(present_labels)*0.5), max(6, len(present_labels)*0.5)))
im = plt.imshow(cm, interpolation='nearest')
plt.colorbar(im)
plt.title("Confusion Matrix (Counts)")
plt.xlabel("Predicted")
plt.ylabel("True")
# 축 라벨
plt.xticks(ticks=np.arange(len(present_labels)), labels=present_labels, rotation=90)
plt.yticks(ticks=np.arange(len(present_labels)), labels=present_labels)
plt.tight_layout()
cm_png_path = os.path.join(OUTPUT_DIR, "confusion_matrix.png")
plt.savefig(cm_png_path, dpi=300)
plt.close()
print(f"[saved] {cm_png_path}")

# (옵션) 정규화 혼동행렬도 저장
cm_norm = cm.astype(float) / (cm.sum(axis=1, keepdims=True) + 1e-12)
cm_norm_csv_path = os.path.join(OUTPUT_DIR, "confusion_matrix_normalized.csv")
pd.DataFrame(cm_norm, index=[f"T_{l}" for l in present_labels],
                columns=[f"P_{l}" for l in present_labels]).to_csv(cm_norm_csv_path, encoding="utf-8-sig")
print(f"[saved] {cm_norm_csv_path}")

plt.figure(figsize=(max(8, len(present_labels)*0.5), max(6, len(present_labels)*0.5)))
im2 = plt.imshow(cm_norm, interpolation='nearest')
plt.colorbar(im2)
plt.title("Confusion Matrix (Row‑Normalized)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks(ticks=np.arange(len(present_labels)), labels=present_labels, rotation=90)
plt.yticks(ticks=np.arange(len(present_labels)), labels=present_labels)
plt.tight_layout()
cm_norm_png_path = os.path.join(OUTPUT_DIR, "confusion_matrix_normalized.png")
plt.savefig(cm_norm_png_path, dpi=300)
plt.close()
print(f"[saved] {cm_norm_png_path}")

[columns] ['Unnamed: 0', 'label', 'text', 'group_id', 'pred_label']
[saved] /content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb/classification_report.csv
[saved] /content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb/confusion_matrix_counts.csv
[saved] /content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb/confusion_matrix.png
[saved] /content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb/confusion_matrix_normalized.csv
[saved] /content/drive/MyDrive/Medic_chatbot/Medic_chatbot/outputs_mb/confusion_matrix_normalized.png
